In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
df = pd.read_csv('../Data/action_log_features.csv')

In [4]:
Q3_2019 = df[(df['at'] >= '2019-07-01') & (df['at'] < '2019-10-01')]
Q4_2019 = df[(df['at'] >= '2019-10-01') & (df['at'] < '2020-01-01')]
Q1_2020 = df[(df['at'] >= '2020-01-01') & (df['at'] < '2020-04-01')]
Q2_2020 = df[(df['at'] >= '2020-04-01') & (df['at'] < '2020-07-01')]
Q3_2020 = df[(df['at'] >= '2020-07-01') & (df['at'] < '2020-10-01')]
Q4_2020 = df[(df['at'] >= '2020-10-01') & (df['at'] < '2021-01-01')]
Q1_2021 = df[(df['at'] >= '2021-01-01') & (df['at'] < '2021-04-01')]
Q2_2021 = df[(df['at'] >= '2021-04-01') & (df['at'] < '2021-07-01')]

In [5]:
Q3_2019.reset_index(inplace = True, drop = True)
Q4_2019.reset_index(inplace = True, drop = True)
Q1_2020.reset_index(inplace = True, drop = True)
Q2_2020.reset_index(inplace = True, drop = True)
Q3_2020.reset_index(inplace = True, drop = True)
Q4_2020.reset_index(inplace = True, drop = True)
Q1_2021.reset_index(inplace = True, drop = True)
Q2_2021.reset_index(inplace = True, drop = True)

In [6]:
red_light = [
    'add_snippet', 'auth_confirm_caution', 'change_contact_status', 'change_password', 'change_service_user_status',
    'clear_account_settings', 'deactivate_workflow', 'download_task_file', 'download_task_files', 'download_template',
    'export_contacts', 'export_template', 'export_workflow', 'remove_account', 'remove_action',
    'remove_contact', 'remove_draft', 'remove_inform', 'remove_role', 'remove_snippet',
    'remove_status', 'remove_workflow', 'reset_password', 'set_new_pass', 'suspend_account',
    'suspend_role', 'two_factor_email'
]

yellow_light = [
    'account_access', 'add_account', 'add_action', 'add_contact', 'add_draft',
    'add_group', 'add_html_template', 'add_role', 'add_status', 'add_workflow',
    'adm_open_settings', 'edit_account', 'edit_account_uname', 'edit_action', 'edit_contact',
    'edit_docx_template', 'edit_draft', 'edit_group', 'edit_html_template', 'edit_profile',
    'edit_role', 'edit_snippet', 'edit_status', 'edit_workflow', 'edit_workflow_group',
    'resend_token', 'token_login', 'unblock_password_reset', 'unsuspend_account', 'unsuspend_role'
]

green_light = [
    'apply_label', 'attach_task_file', 'attach_template', 'clone_workflow', 'create_task',
    'create_task', 'import_contacts', 'import_workflow', 'login','logout', 'make_available',
    'make_unavailable', 'open_account', 'open_action', 'open_contact', 'open_docx_template',
    'open_group', 'open_html_template', 'open_incident_log', 'open_new_account', 'open_new_action',
    'open_new_contact', 'open_new_docx_template', 'open_new_group', 'open_new_html_template', 'open_new_role',
    'open_new_snippet', 'open_new_status', 'open_new_workflow', 'open_perform_action', 'open_profile',
    'open_role', 'open_snippet', 'open_status', 'open_task', 'open_workflow',
    'open_workflow_group', 'perform_action', 'reset_req', 'set_action_state', 'set_status_state'
]

In [7]:
# Q2_2021[Q2_2021.columns[2:]].sum()[Q2_2021[Q2_2021.columns[2:]].sum() == 0].index

In [8]:
# Q2_2021[Q2_2021.columns[2:]].sum().sort_values()

In [16]:
fig = go.Figure(go.Line(x=red_light, y=np.array(Q3_2019[red_light].sum()).astype(int), name='Q3_2019'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q4_2019[red_light].sum()).astype(int), name='Q4_2019'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q1_2020[red_light].sum()).astype(int), name='Q1_2020'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q2_2020[red_light].sum()).astype(int), name='Q2_2020'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q3_2020[red_light].sum()).astype(int), name='Q3_2020'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q4_2020[red_light].sum()).astype(int), name='Q4_2020'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q1_2021[red_light].sum()).astype(int), name='Q1_2021'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q2_2021[red_light].sum()).astype(int), name='Q2_2021'))
fig.show()

In [10]:
fig = go.Figure(go.Line(x=red_light, y=np.array(Q3_2019[red_light].max()).astype(int), name='Q3_2019'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q4_2019[red_light].max()).astype(int), name='Q4_2019'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q1_2020[red_light].max()).astype(int), name='Q1_2020'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q2_2020[red_light].max()).astype(int), name='Q2_2020'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q3_2020[red_light].max()).astype(int), name='Q3_2020'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q4_2020[red_light].max()).astype(int), name='Q4_2020'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q1_2021[red_light].max()).astype(int), name='Q1_2021'))
fig.add_trace(go.Line(x=red_light, y=np.array(Q2_2021[red_light].max()).astype(int), name='Q2_2021'))
fig.show()

In [11]:
fig = go.Figure(go.Line(x=yellow_light, y=np.array(Q3_2019[yellow_light].sum()).astype(int), name='Q3_2019'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q4_2019[yellow_light].sum()).astype(int), name='Q4_2019'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q1_2020[yellow_light].sum()).astype(int), name='Q1_2020'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q2_2020[yellow_light].sum()).astype(int), name='Q2_2020'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q3_2020[yellow_light].sum()).astype(int), name='Q3_2020'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q4_2020[yellow_light].sum()).astype(int), name='Q4_2020'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q1_2021[yellow_light].sum()).astype(int), name='Q1_2021'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q2_2021[yellow_light].sum()).astype(int), name='Q2_2021'))
fig.show()

In [12]:
fig = go.Figure(go.Line(x=yellow_light, y=np.array(Q3_2019[yellow_light].max()).astype(int), name='Q3_2019'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q4_2019[yellow_light].max()).astype(int), name='Q4_2019'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q1_2020[yellow_light].max()).astype(int), name='Q1_2020'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q2_2020[yellow_light].max()).astype(int), name='Q2_2020'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q3_2020[yellow_light].max()).astype(int), name='Q3_2020'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q4_2020[yellow_light].max()).astype(int), name='Q4_2020'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q1_2021[yellow_light].max()).astype(int), name='Q1_2021'))
fig.add_trace(go.Line(x=yellow_light, y=np.array(Q2_2021[yellow_light].max()).astype(int), name='Q2_2021'))
fig.show()

In [13]:
fig = go.Figure(go.Line(x=green_light, y=np.array(Q3_2019[green_light].sum()).astype(int), name='Q3_2019'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q4_2019[green_light].sum()).astype(int), name='Q4_2019'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q1_2020[green_light].sum()).astype(int), name='Q1_2020'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q2_2020[green_light].sum()).astype(int), name='Q2_2020'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q3_2020[green_light].sum()).astype(int), name='Q3_2020'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q4_2020[green_light].sum()).astype(int), name='Q4_2020'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q1_2021[green_light].sum()).astype(int), name='Q1_2021'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q2_2021[green_light].sum()).astype(int), name='Q2_2021'))
fig.show()

In [14]:
fig = go.Figure(go.Line(x=green_light, y=np.array(Q3_2019[green_light].max()).astype(int), name='Q3_2019'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q4_2019[green_light].max()).astype(int), name='Q4_2019'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q1_2020[green_light].max()).astype(int), name='Q1_2020'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q2_2020[green_light].max()).astype(int), name='Q2_2020'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q3_2020[green_light].max()).astype(int), name='Q3_2020'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q4_2020[green_light].max()).astype(int), name='Q4_2020'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q1_2021[green_light].max()).astype(int), name='Q1_2021'))
fig.add_trace(go.Line(x=green_light, y=np.array(Q2_2021[green_light].max()).astype(int), name='Q2_2021'))
fig.show()

In [99]:
Q4_2020_features = Q4_2020[Q4_2020[['download_task_files', 'auth_confirm_caution', 'remove_inform']].sum(1) != 0].reset_index(drop = True)

In [100]:
users = list(map(lambda i: i[0] + ' ' + str(i[1]), zip(Q4_2020_features['at'].tolist(), Q4_2020_features['account'].tolist())))

In [101]:
# users

In [102]:
Q4_2020_features = Q4_2020_features[['download_task_files', 'auth_confirm_caution', 'remove_inform']]

In [103]:
x = Q4_2020_features.values
scaler = preprocessing.MinMaxScaler()
x_scaled = scaler.fit_transform(x)
X_norm = pd.DataFrame(x_scaled)

In [104]:
pca = PCA(n_components = 2)
reduced = pd.DataFrame(pca.fit_transform(X_norm))

In [48]:
# calculate the distance from each point to its closest neighbor
nn = NearestNeighbors(n_neighbors = 2)

# fit the nearest neighbor
nbrs = nn.fit(reduced)

# returns two arrays - distance to the closest n_neighbors points and index for each point
distances, indices = nbrs.kneighbors(reduced)
print(distances.shape)

# sort the distance and plot it
distances = np.sort(distances, axis=0)
distances = distances[:,1]
px.line(distances)

(21083, 2)


In [105]:
# train the model using DBSCAN
db = DBSCAN(eps=0.03, min_samples=15)

# prediction for dbscan clusters
db_clusters = db.fit_predict(reduced)

In [106]:
list(set(db_clusters))

[0, -1]

In [107]:
reduced['cluster'] = db_clusters
reduced['user'] = users
reduced.columns = ['x', 'y', 'cluster', 'name']
reduced.head()

,x,y,cluster,name
0,-0.012691,-0.017887,0,2020-10-06 2.0
1,0.004993,0.173071,0,2020-10-07 2.0
2,-0.008902,0.023032,0,2020-10-08 2.0
3,-0.010165,0.009392,0,2020-10-09 2.0
4,-0.012691,-0.017887,0,2020-10-12 2.0


In [108]:
px.scatter(reduced, x='x', y='y', color="cluster",
           color_continuous_scale=px.colors.diverging.Portland)

In [109]:
reduced[(reduced['cluster'] == -1)]

,x,y,cluster,name
423,0.892609,-0.060618,-1,2020-11-24 15.0
842,0.680412,0.000343,-1,2020-11-30 22.0
1302,0.078255,0.964185,-1,2020-12-23 31.0
1978,0.423072,-0.017050,-1,2020-12-24 53.0
3170,0.487544,-0.036790,-1,2020-11-10 87.0
3512,0.499770,-0.010320,-1,2020-11-23 96.0
4054,0.617751,-0.062629,-1,2020-11-11 109.0
4162,0.041458,0.362287,-1,2020-12-01 111.0
4179,0.059691,0.456895,-1,2020-12-24 111.0
4180,0.064361,0.814146,-1,2020-12-25 111.0


In [112]:
test = Q4_2020[Q4_2020[['download_task_files', 'auth_confirm_caution', 'remove_inform']].sum(1) != 0].reset_index(drop = True)
test = test[['at', 'account', 'download_task_files', 'auth_confirm_caution', 'remove_inform']]

In [133]:
px.line(test[test['account'] == 361], x='at', y=test.columns.tolist()[2:])